# Data Preprocessing
## 1. 결손값, NaN, Null값 처리
- feature 값 중 null값이 얼마 되지 않는 경우 
    - mean값 등으로 처리
- Null값이 대부분인 경우  
    - feature drop하는 것이 더 나음 
- null값이 일정 수준 이상인 경우   
    - feature의 중요도에 따라 결정  
    - 업무 로직에 따라서 정밀한 대체값 결정 (그냥 평균으로 하면 왜곡 심해질 수 있음)  



## 2. 문자열값 처리해주기
- 문자열을 입력 값으로 허용하지 X
- 모든 문자열 값을 encoding해서 숫자형으로 변환
- 종류
    - 카테고리형 feature
        - feature 벡터화 기법을 통해 벡터화
        - 불필요한 feature면 drop
    - text형 feature



## Data Encoding
### 1) Label Encoding
- category feature을 코드형 숫자 값으로 변환
- LabelEncoder class이용
- 문자열이 어떤값으로 Encoding된건 지 알기 위해 classes_ 속성값 이용
- Encoding된 값을 다시 Decoding 하려면 inverse_transform() 사용
- Ex) Tv : 0 , 냉장고 :1 , 선풍기 :2로 할당 


#### 특징
- ML Algorithm에 적용할 경우 예측 성능 저하
    - 숫자 값이 크고 작음에 대한 특성이 작용하기 때문
    - 숫자 값이 더 큰 경우에 ML에서 가중치를 부여함
    - Label Encoding은 선형 회귀와 같은 algorithm에는 적용하지 X
    - Tree 계열의 ML algorithm에서는 사용가능
     



#### 순서
1. LabelEncoder을 객체로 생성
2. fit(), transform()으로 label Encoding 수행

In [4]:
from sklearn.preprocessing import LabelEncoder

In [2]:
items=['TV','냉장고','전자렌지','컴터','선풍기','선풍기','믹서','믹서']

In [4]:
encoder=LabelEncoder()
encoder.fit(items)
labels=encoder.transform(items)

In [5]:
labels

array([0, 1, 4, 5, 3, 3, 2, 2], dtype=int64)

In [9]:
encoder.classes_

array(['TV', '냉장고', '믹서', '선풍기', '전자렌지', '컴터'], dtype='<U4')

In [11]:
#Encoding된 값을 다시 Decoding 
encoder.inverse_transform([4,5,2,0])

array(['전자렌지', '컴터', '믹서', 'TV'], dtype='<U4')

## 2) One - Hot Encoding 
- 값의 유형에 따라 새로운 feature 추가
- 고유값에 해당하는 column에만 1을 표시, 나머지는 0
- 모든 문자열의 값이 숫자형으로 이미 변환되어 있어야지 사용 가능 
- 입력값으로 2차원 data가 필요

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
items=['TV','냉장고','전자렌지','컴터','선풍기','선풍기','믹서','믹서']

In [6]:
# 먼저 숫자값으로 변환 위해 LabelEncoder로 변환
encoder =LabelEncoder()
encoder.fit(items)
labels=encoder.transform(items)

In [7]:
labels

array([0, 1, 4, 5, 3, 3, 2, 2], dtype=int64)

In [8]:
# 2차원 data로 변환해주기
labels=labels.reshape(-1,1)

In [9]:
labels

array([[0],
       [1],
       [4],
       [5],
       [3],
       [3],
       [2],
       [2]], dtype=int64)

In [10]:
# One hot encoding
oh_encoder=OneHotEncoder()
oh_encoder.fit(labels)
oh_labels=oh_encoder.transform(labels)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
# one- hot encoding data
oh_labels.toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

### get_dummies() 
- One hot encoding 더 쉽게 지원하는 API
- 문자열을 숫자형으로 변환할 필요 없이 바로 변환 가능

In [13]:
import pandas as pd

In [15]:
df=pd.DataFrame({'item':['TV','냉장고','전자렌지','컴터','선풍기','선풍기','믹서','믹서']})

In [16]:
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자렌지,item_컴터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## Feature Scaling 
### (1) 표준화 (Standarization)
- data Feature값이 0 or 1인 가우시안 정규 분포 가진 값으로 변환
- (xi-mean(x))/std(x)

### (2) 정규화 (Normalization)
- 서로 다른 Feature의 크기를 통일하기 위해 크기 변환
- 개별 데이터의 크기를 똑같은 단위로 변경
- Ex) 거리 변수 0~100 , 금액 0~10000000이면 0~1사이의 값으로 변환



### StandardScaler
- 표준화 지원하는 class
- feature를 평균 0, 분산 1로 변환
- RBF 커널을 이용한 SVM, 선형회귀, 로지스틱 회귀는 데이터가 가우시안 분포를 가지고 있다고 구현했음
    - 사전에 표준화 필요
- transform()의 결과 ndarray형태로 반환됌
    - DataFrame으로 바꿔줌

In [25]:
from sklearn.datasets import load_iris
import pandas as pd

In [19]:
iris=load_iris()
iris_df=pd.DataFrame(data=iris.data, columns=iris.feature_names)

In [20]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
scaler=StandardScaler()
scaler.fit(iris_df)
iris_scaled=scaler.transform(iris_df)

In [26]:
iris_df_scaled=pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

In [27]:
iris_df_scaled.mean()

sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

In [28]:
iris_df_scaled.var()

sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64

### Result
모든 column들에 대한 평균은 0에, 분산은 1에 가까운 값으로 변환

### MinMaxScaler
- data값을 0~1사이의 값으로 변환
    - 음수값 있으면 -1~1 사이의 값으로 변환
- 데이터의 분포가 가우시안이 아닌 경우 사용

In [30]:
from sklearn.preprocessing import MinMaxScaler

In [32]:
scaler=MinMaxScaler()
scaler.fit(iris_df)
iris_scaled=scaler.transform(iris_df)

In [34]:
iris_df_scaled=pd.DataFrame(data=iris_scaled,columns=iris.feature_names)

In [35]:
iris_df_scaled.min()

sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

In [37]:
iris_df_scaled.max()

sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64

### Result
모든 feature가 0~1값으로 변환됌

## Train & Test Data의 스케일링 변환 
- Function Info
    - fit() : data변환을 위한 기준 정보 설정 적용
    - transform() : 설정된 정보 이용해 데이터 변환
    - fit_transform() : 2개를 한번에 수행
- 주의점
    - 학습데이터로 fit, transform시키면 test data로는 다시 fit, transform수행하지 않음
    - train data로 fit이 적용된 스케일링 기준 정보를 그대로 사용함 
    - test data로 다시 새로운 스케일링 기준 정보를 만들면 올바른 예측 결과 도출 X
#### 즉, test data에 scale변환 할 시, fit()호출 X & transform()만으로 변환해야함
#### 1. 가능하면 전체 data의 scale변환 적용한 뒤 train/test data로 분리

In [16]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [5]:
train_arr=np.arange(0,11).reshape(-1,1)
test_arr=np.arange(0,6).reshape(-1,1)

In [6]:
# 올바르게 하는 법
scaler=MinMaxScaler()
scaler.fit(train_arr)
train_scaled=scaler.transform(train_arr)

In [10]:
# 원본 train_arr
train_arr.reshape(-1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [11]:
# scale된 train_arr
train_scaled.reshape(-1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [13]:
# test data에 scale변환 할 시, fit()호출 X & transform()만으로 변환해야함
test_scaled=scaler.transform(test_arr)

In [14]:
# 원본 test data
test_arr.reshape(-1)

array([0, 1, 2, 3, 4, 5])

In [15]:
# scale된 test data
test_scaled.reshape(-1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5])